<a href="https://colab.research.google.com/github/parthag1201/Authentication_google/blob/main/FirstCrew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing the dependencies
!pip install crewai langchain-community pydantic
!pip install 'crewai[tools]'
!pip install 'crewai[CrewStructuredTool]'
!pip install -U duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.1 MB/s eta 0:00:

In [2]:
# Using required packages

from crewai import Crew, Task, Agent, Process, LLM
from pydantic import BaseModel
from crewai.tools.structured_tool import CrewStructuredTool
from crewai.tools import tool
from langchain_community.tools import BraveSearch
from langchain.tools import Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.llms import Ollama  # If we use Ollama instead of Gemini
from google.colab import userdata # For API Secret

In [3]:
# Choose LLM

llm = LLM(
    model='gemini/gemini-2.0-flash',  # Use 'vertex_ai' as the provider for Google's Vertex AI
    api_key=userdata.get('API_KEY')
)

In [4]:
# Define Web Search Engine

@tool("duckduckgo_search_tool")
def duckduckgo_search_wrapper(query: str) -> str:
    """Searches the web using DuckDuckGo."""
    search_tool = DuckDuckGoSearchRun(region="pt-br")
    result = search_tool.run(query)
    return result

In [5]:
# Define agents

web_researcher_agent = Agent(
    role="Web Research Specialist",
    goal=(
        "To find the most recent, impactful, and relevant about {topic}. This includes identifying "
        "key use cases, challenges, and statistics to provide a foundation for deeper analysis."
    ),
    backstory=(
        "You are a former investigative journalist known for your ability to uncover technology breakthroughs "
        "and market insights. With years of experience, you excel at identifying actionable data and trends."
    ),
    tools=[],  # Use the decorated function directly
    llm=llm,
    verbose=True
)

trend_analyst_agent = Agent(
    role="Insight Synthesizer",
    goal=(
        "To analyze research findings, extract significant trends, and rank them by industry impact, growth potential, "
        "and uniqueness. Provide actionable insights for decision-makers."
    ),
    backstory=(
        "You are a seasoned strategy consultant who transitioned into {topic} analysis. With an eye for patterns, "
        "you specialize in translating raw data into clear, actionable insights."
    ),
    tools=[],
    llm=llm,
    verbose=True
)

report_writer_agent = Agent(
    role="Narrative Architect",
    goal=(
        "To craft a detailed, professional report that communicates research findings and analysis effectively. "
        "Focus on clarity, logical flow, and engagement."
    ),
    backstory=(
        "Once a technical writer for a renowned journal, you are now dedicated to creating industry-leading reports. "
        "You blend storytelling with data to ensure your work is both informative and captivating."
    ),
    tools=[],
    llm=llm,
    verbose=True
)

proofreader_agent = Agent(
    role="Polisher of Excellence",
    goal=(
        "To refine the report for grammatical accuracy, readability, and formatting, ensuring it meets professional "
        "publication standards."
    ),
    backstory=(
        "An award-winning editor turned proofreader, you specialize in perfecting written content. Your sharp eye for "
        "detail ensures every document is flawless."
    ),
    tools=[],
    llm=llm,
    verbose=True
)

manager_agent = Agent(
    role="Workflow Maestro",
    goal=(
        "To coordinate agents, manage task dependencies, and ensure all outputs meet quality standards. Your focus "
        "is on delivering a cohesive final product through efficient task management."
    ),
    backstory=(
        "A former project manager with a passion for efficient teamwork, you ensure every process runs smoothly, "
        "overseeing tasks and verifying results."
    ),
    tools=[],
    llm=llm,
    verbose=True
)

In [6]:
# Define tasks

web_research_task = Task(
    description=(
        "Conduct web-based research to identify 5-7 of the {topic}. Focus on key use cases. "
    ),
    expected_output=(
        "A structured list of 5-7 {topic}."
    )
)

trend_analysis_task = Task(
    description=(
        "Analyze the research findings to rank {topic}. "
    ),
    expected_output=(
        "A table ranking trends by impact, with concise descriptions of each trend."
    )
)

report_writing_task = Task(
    description=(
        "Draft report summarizing the findings and analysis of {topic}. Include sections for "
        "Introduction, Trends Overview, Analysis, and Recommendations."
    ),
    expected_output=(
        "A structured, professional draft with a clear flow of information. Ensure logical organization and consistent tone."
    )
)

proofreading_task = Task(
    description=(
        "Refine the draft for grammatical accuracy, coherence, and formatting. Ensure the final document is polished "
        "and ready for publication."
    ),
    expected_output=(
        "A professional, polished report free of grammatical errors and inconsistencies. Format the document for "
        "easy readability."
    )
)

In [7]:
# Create Crew.ai instance

crew = Crew(
    agents=[web_researcher_agent, trend_analyst_agent, report_writer_agent, proofreader_agent],
    tasks=[web_research_task, trend_analysis_task, report_writing_task, proofreading_task],
    process=Process.hierarchical,
    manager_agent=manager_agent,
    verbose=True
)

In [8]:
# Start Crew.ai by giving the required input

crew_output = crew.kickoff(inputs={"topic": "Important of AI in Supply Chain Management (Specifically for Food product based - FMCG Companies)"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 1697984c-7970-4234-9950-87b6070216a2                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 2bfb42d7-972b-4722-9e1a-c2b1f350fe56
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 2bfb42d7-972b-4722-9e1a-c2b1f350fe56
       Status: Executing Task...
    └── 🤖 Agent: Workflow Maestro
            Status: In Progress

# Agent: Workflow Maestro
## Task: Conduct web-based research to identify 5-7 of the Important of AI in Supply Chain Management (Specifically for Food product based - FMCG Companies). Focus on key use cases. 


🚀 Crew: crew
└── 📋 Task: 2bfb42d7-972b-4722-9e1a-c2b1f350fe56
       Status: Executing Task...
    └── 🤖 Agent: Workflow Maestro
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: 2bfb42d7-972b-4722-9e1a-c2b1f350fe56
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (1)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
├── 📋 Task: 8762f99f-37f6-4260-ade7-f0565f355d51
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (2)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
├── 📋 Task: 2450498b-ac1c-4761-8d8f-efaa09fb2349
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (3)
│   ├── 🤖 Agent: Narrative Architect
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (4)
│   ├── 🤖 Agent: Web Research Specialist
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (5)
│   ├── 🤖 Agent: Insight Synthesizer
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (6)
│   ├── 🤖 Agent: Narrative Architect
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (7)
│   ├── 🤖 Agent: Polisher of Excellence
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (8)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
└── 📋 Task: fc671e7b-353d-4d6d-be1c-f3a8c4a982c9
       Status: Executing Task...
    ├── 🤖 Agent: Workflow Maestro
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (9)
    ├── 🤖 Agent: Polisher of Excellence
    │       Status: ✅ Completed
    ├── 🤖 Agent: Workflow Maestro
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (10)
    ├── 🤖 Agent: Polisher of Excellence
    │       Status: ✅ Completed
    │   └── 🔧 Using Ask question to coworker (1)
    ├── 🤖 Agent: Insight Synthesizer
    │       Status: ✅ Completed
    ├── 🤖 Agent: Workflow Maestro
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (11)
    ├── 🤖 Agent: Polisher of Excellence
    │       Status: ✅ Completed
    │   └── 🔧 Using Delegate work to coworker (12)
    ├── 🤖 Agent: Insight Synthesizer
    │       Status: ✅ Completed
    │   └── 🔧 Using Delegate work to coworker (13)
    └── 🤖 Agent: Workflow Maestro
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 2bfb42d7-972b-4722-9e1a-c2b1f350fe56
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (1)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
├── 📋 Task: 8762f99f-37f6-4260-ade7-f0565f355d51
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (2)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
├── 📋 Task: 2450498b-ac1c-4761-8d8f-efaa09fb2349
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (3)
│   ├── 🤖 Agent: Narrative Architect
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (4)
│   ├── 🤖 Agent: Web Research Specialist
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (5)
│   ├── 🤖 Agent: Insight Synthesizer
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (6)
│   ├── 🤖 Agent: Narrative Architect
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (7)
│   ├── 🤖 Agent: Polisher of Excellence
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (8)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
└── 📋 Task: fc671e7b-353d-4d6d-be1c-f3a8c4a982c9
       Status: Executing Task...
    ├── 🤖 Agent: Workflow Maestro
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (9)
    ├── 🤖 Agent: Polisher of Excellence
    │       Status: ✅ Completed
    ├── 🤖 Agent: Workflow Maestro
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (10)
    ├── 🤖 Agent: Polisher of Excellence
    │       Status: ✅ Completed
    │   └── 🔧 Using Ask question to coworker (1)
    ├── 🤖 Agent: Insight Synthesizer
    │       Status: ✅ Completed
    ├── 🤖 Agent: Workflow Maestro
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (11)
    ├── 🤖 Agent: Polisher of Excellence
    │       Status: ✅ Completed
    │   └── 🔧 Using Delegate work to coworker (12)
    ├── 🤖 Agent: Insight Synthesizer
    │       Status: ✅ Completed
    │   └── 🔧 Using Delegate work to coworker (13)
    └── 🤖 Agent: Workflow Maestro
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 2bfb42d7-972b-4722-9e1a-c2b1f350fe56
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (1)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
├── 📋 Task: 8762f99f-37f6-4260-ade7-f0565f355d51
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (2)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
├── 📋 Task: 2450498b-ac1c-4761-8d8f-efaa09fb2349
│      Assigned to: Workflow Maestro
│      Status: ✅ Completed
│   ├── 🤖 Agent: Workflow Maestro
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (3)
│   ├── 🤖 Agent: Narrative Architect
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (4)
│   ├── 🤖 Agent: Web Research Specialist
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (5)
│   ├── 🤖 Agent: Insight Synthesizer
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (6)
│   ├── 🤖 Agent: Narrative Architect
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (7)
│   ├── 🤖 Agent: Polisher of Excellence
│   │       Status: ✅ Completed
│   │   └── 🔧 Using Delegate work to coworker (8)
│   └── 🤖 Agent: Workflow Maestro
│           Status: ✅ Completed
└── 📋 Task: fc671e7b-353d-4d6d-be1c-f3a8c4a982c9
       Assigned to: Workflow Maestro
       Status: ✅ Completed
    ├── 🤖 Agent: Workflow Maestro
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (9)
    ├── 🤖 Agent: Polisher of Excellence
    │       Status: ✅ Completed
    ├── 🤖 Agent: Workflow Maestro
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (10)
    ├── 🤖 Agent: Polisher of Excellence
    │       Status: ✅ Completed
    │   └── 🔧 Using Ask question to coworker (1)
    ├── 🤖 Agent: Insight Synthesizer
    │       Status: ✅ Completed
    ├── 🤖 Agent: Workflow Maestro
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (11)
    ├── 🤖 Agent: Polisher of Excellence
    │       Status: ✅ Completed
    │   └── 🔧 Using Delegate work to coworker (12)
    ├── 🤖 Agent: Insight Synthesizer
    │       Status: ✅ Completed
    │   └── 🔧 Using Delegate work to coworker (13)
    └── 🤖 Agent: Workflow Maestro
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: fc671e7b-353d-4d6d-be1c-f3a8c4a982c9                                                                     │
│  Agent: Workflow Maestro                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 1697984c-7970-4234-9950-87b6070216a2                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [9]:
# Output

print(crew_output)

```
**AI in Supply Chain Management for Food Product-Based FMCG Companies**

**Introduction**

The fast-moving consumer goods (FMCG) industry, particularly those focused on food products, faces unique supply chain challenges. These challenges include stringent shelf-life requirements, complex distribution networks, and fluctuating consumer demand. Artificial intelligence (AI) offers transformative potential to optimize these processes, enhancing efficiency, reducing waste, and improving overall profitability. This report explores current AI trends in supply chain management for food product-based FMCG companies, analyzes their impact, and provides actionable recommendations for implementation.

**Trends Overview**

Several key AI-driven trends are reshaping supply chain management in the food FMCG sector:

1.  **Demand Forecasting:** AI algorithms analyze historical sales data, market trends, weather patterns, and even social media sentiment to predict future demand with greater accura